# Reading PHYSLITE files using the `PHYSLITESchema`

In [ ]:
import coffea
print("coffea version: ", coffea.__version__)
import awkward as ak
print("awkward version: ", ak.__version__)
from coffea.nanoevents import NanoEventsFactory, PHYSLITESchema
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Hgg -> 4l sample

# local
# file_path = '/MyMachine/Downloads/DAOD_PHYSLITE.38191712._000001.pool.root.1'

# stream
file_path = "root://eospublic.cern.ch//eos/opendata/atlas/rucio/mc20_13TeV/DAOD_PHYSLITE.38191712._000001.pool.root.1"

In [ ]:
# load in memory

def filter_name(name):
    '''
    Load only the properties/variables needed.
    '''
    return name in [
        "EventInfoAuxDyn.mcEventWeights",
        
        "AnalysisElectronsAuxDyn.pt",
        "AnalysisElectronsAuxDyn.eta",
        "AnalysisElectronsAuxDyn.phi",
        "AnalysisElectronsAuxDyn.m",
        "AnalysisElectronsAuxDyn.DFCommonElectronsLHLoose",
        "AnalysisElectronsAuxDyn.charge",
        
        "AnalysisMuonsAuxDyn.pt",
        "AnalysisMuonsAuxDyn.eta",
        "AnalysisMuonsAuxDyn.phi",
        "AnalysisMuonsAuxDyn.m",
        "AnalysisMuonsAuxDyn.quality",
        
        "AnalysisJetsAuxDyn.pt",
        "AnalysisJetsAuxDyn.eta",
        "AnalysisJetsAuxDyn.phi",
        "AnalysisJetsAuxDyn.m",
        
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pb",
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pc",
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pu",
    ]

events = NanoEventsFactory.from_root(
    {file_path: "CollectionTree"}, # all the event properties are stored in the TTree called CollectionTree
    schemaclass=PHYSLITESchema, # tell NanoEventsFactory.from_root that you read a PHYSLITE file
    uproot_options=dict(filter_name=filter_name)
).events()

In [ ]:
events.fields

In [ ]:
events.Electrons.fields

In [ ]:
events.Electrons.compute()

## Slicing

In [ ]:
# define boolean array
selection_2e = ak.num(events.Electrons, axis=-1) > 1

# print the selected events
events[selection_2e].compute()

In [ ]:
# print the second of the selected events
events[selection_2e][1].compute()

In [ ]:
# print the leading electron pt of the selected events
events.Electrons[selection_2e][:, 0].pt.compute() # in MeV
# events[selection_2e][:, "Electrons"][:, "pt"][:, 0].compute() # another uglier way to write this

In [ ]:
# distance between leading and sub-leading electron in every event
dr = events.Electrons[selection_2e][:, 0].delta_r(events.Electrons[selection_2e][:, 1])

# compute and plot
plt.hist(dr.compute(), bins=50, range=(0, 5))
plt.xlabel(r"$\Delta R(e_0, e_1)$")
plt.ylabel("Events")
plt.show()

Let's now use the schema to calculate the invariant mass of a group of particles. In particular, calculate and plot the invariant mass of the four leading electrons of each event.

In [ ]:
# your code here

## Element Links

In [ ]:
# naive ElementLinking
def calculate_jets_DL1dv01(events):
    
    BTagging = events.BTagging_AntiKt4EMPFlow
    
    f_c = 0.018
    DL1dv01 = BTagging.DL1dv01_pb/(f_c*BTagging.DL1dv01_pc + (1-f_c)*BTagging.DL1dv01_pu)
    DL1dv01 = np.log(DL1dv01)

    return DL1dv01

In [ ]:
# assign new variables to the arrays
events['Jets', 'DL1dv01'] = calculate_jets_DL1dv01(events)

events.Jets.fields

In [ ]:
events.Jets.DL1dv01.compute()